In [1]:
# Importar librerias y configurar logging
import pandas as pd
from pathlib import Path
import tabula
from unidecode import unidecode
import logging
logging.basicConfig(level=logging.DEBUG, filename='log.log',  format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger = logging.getLogger(__name__)

In [2]:
# Listar archivos pdf
mi_path = 'fichas/'
lista_pdfs = list(Path(mi_path).glob('**/*.pdf'))

In [3]:
# Extraer tablas
tablas = []
for pdf in lista_pdfs:
    try:
        tabla = tabula.read_pdf(pdf, pages='all', guess=False, lattice=True)
        tablas.append(tabla)
    except Exception as e:
        tablas.append(None)
        logger.error(f'Error al extraer tablas de {pdf}')
        logger.error(e)
        continue

In [4]:
# Limpiar tablas
nombres_pdfs = [pdf.name.replace('.pdf', '') for pdf in lista_pdfs]
tablas_clean = []
for idx, tabla in enumerate(tablas):
    try:
        tabla_x = tabla[4] if tabla[4].columns.str.contains('UNSPS').any() else tabla[5] if tabla[5].columns.str.contains('UNSPS').any() else tabla[6]
        tabla_x.columns = tabla_x.columns.str.replace('\r', '', regex=True).str.replace(' ', '').to_series().apply(unidecode)
        tabla_x = tabla_x.drop(tabla_x.columns[-1], axis=1)
        tabla_x.replace('\r', ' ', regex=True, inplace=True)
        tabla_x.drop(tabla_x.index[-1], inplace=True)
        tabla_x.insert(0, 'cuce', nombres_pdfs[idx])
        tabla_x.insert(1, 'list_n', idx)
        tablas_clean.append(tabla_x)
    except Exception as e:
        tablas_clean.append(None)
        logger.error(f'Error al procesar tablas de la lista: {idx}: {nombres_pdfs[idx]}')
        logger.error(e)
        continue

In [5]:
# Unir tablas
df_clean = pd.concat(tablas_clean, ignore_index=True)

In [6]:
# Exportar a csv
df_clean.to_csv('../../fichas/fichas_nuevas.csv', index=False)

In [7]:
import pickle
with open('../../fichas/fichas_nuevas.pkl', 'wb') as f:
    pickle.dump(tablas, f)

with open('../../fichas/lista_pdfs.pkl', 'wb') as f:
    pickle.dump(lista_pdfs, f)

In [8]:
len(lista_pdfs) == len(tablas) == len(tablas_clean)

True